In [1]:
!pip install -q tokenizers
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 11.5 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional, Dropout, LSTM, Embedding, Masking
from tokenizers import Tokenizer, decoders, models, normalizers, pre_tokenizers, trainers, processors
from sklearn.model_selection import train_test_split
import pickle
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path='/content/drive/MyDrive/praca/c_train'
data=pd.read_csv(f'{path}/c_train.csv', sep='\t', encoding='utf-8').drop_duplicates(subset=['nl']).reset_index(drop=True)
x = data.nl.to_numpy()
y = data.code.to_numpy()

In [7]:
nl_tokenizer = Tokenizer(models.WordPiece(unk_token='[UNK]'))
nl_tokenizer.normalizer = normalizers.BertNormalizer(clean_text = False)
nl_tokenizer.pre_tokenizer = pre_tokenizers.WhitespaceSplit()
nl_tokenizer.decoder = decoders.WordPiece(prefix='##')
trainer = trainers.WordPieceTrainer(
    vocab_size=8000,
    show_progress=True,
    special_tokens=['[PAD]', '[UNK]', '[CLS]', '[SEP]'],
    continuing_subword_prefix='##'
)
nl_tokenizer.train_from_iterator(x, trainer=trainer)
nl_tokenizer.post_processor = processors.BertProcessing(('[CLS]',nl_tokenizer.token_to_id('[CLS]')),('[SEP]',nl_tokenizer.token_to_id('[SEP]')))

In [8]:
code_tokenizer = Tokenizer(models.WordPiece(unk_token='[UNK]'))
code_tokenizer.normalizer = normalizers.BertNormalizer(clean_text = False)
code_tokenizer.pre_tokenizer = pre_tokenizers.WhitespaceSplit()
code_tokenizer.decoder = decoders.WordPiece(prefix='##')
trainer = trainers.WordPieceTrainer(
    vocab_size=8000,
    show_progress=True,
    special_tokens=['[PAD]', '[UNK]', '[CLS]', '[SEP]'],
    continuing_subword_prefix='##'
)
code_tokenizer.train_from_iterator(y, trainer=trainer)
code_tokenizer.post_processor = processors.BertProcessing(('[CLS]',code_tokenizer.token_to_id('[CLS]')),('[SEP]',code_tokenizer.token_to_id('[SEP]')))

In [9]:
nl_tokenizer.save(f'{path}/nl_tokenizer.json')
code_tokenizer.save(f'{path}/code_tokenizer.json')

In [ ]:
filter_x = np.array(list(map(lambda item: item.ids, nl_tokenizer.encode_batch(x))))
filter_y = np.array(list(map(lambda item: item.ids, code_tokenizer.encode_batch(y))))
length=400
nl_tokenizer.enable_padding()
code_tokenizer.enable_padding()
x=x[(np.array([len(i) for i in filter_y])<length) & (np.array([len(i) for i in filter_x])<length)]
y=y[(np.array([len(i) for i in filter_y])<length) & (np.array([len(i) for i in filter_x])<length)]
del filter_x
del filter_y
x_tokenized=np.array(list(map(lambda t: t.ids, nl_tokenizer.encode_batch(x))))
y_tokenized=np.array(list(map(lambda t: t.ids, code_tokenizer.encode_batch(y))))
x_tokenized.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x_tokenized, y_tokenized, test_size=0.2, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

np.save(f'{path}/X_train.npy', X_train)
np.save(f'{path}/y_train.npy', y_train)
np.save(f'{path}/X_test.npy', X_test)
np.save(f'{path}/y_test.npy', y_test)
np.save(f'{path}/X_val.npy', X_val)
np.save(f'{path}/y_val.npy', y_val)

input_vocab_size=nl_tokenizer.get_vocab_size()
output_vocab_size=code_tokenizer.get_vocab_size()
input_vocab_size,output_vocab_size

(8000, 8000)

In [ ]:
input_length=X_train.shape[1]
output_length=y_train.shape[1]
model = Sequential([
Masking(mask_value=0,input_shape=(input_length, 1)),
Bidirectional(LSTM(input_length, return_sequences=False)),
RepeatVector(output_length),
LSTM(512, return_sequences=True),
LSTM(1024, return_sequences=True),
Dropout(0.2),
TimeDistributed(Dense(1024)),
Dropout(0.2),
TimeDistributed(Dense(units=output_vocab_size))
])
model.compile()
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking_4 (Masking)         (None, 398, 1)            0         
                                                                 
 bidirectional_4 (Bidirectio  (None, 796)              1273600   
 nal)                                                            
                                                                 
 repeat_vector_4 (RepeatVect  (None, 399, 796)         0         
 or)                                                             
                                                                 
 lstm_15 (LSTM)              (None, 399, 512)          2680832   
                                                                 
 lstm_16 (LSTM)              (None, 399, 1024)         6295552   
                                                                 
 dropout_8 (Dropout)         (None, 399, 1024)        

In [ ]:
def loss_function(x, y):
  cross_entropy = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
  loss = cross_entropy(y_true=y, y_pred=x)
  mask = tf.logical_not(tf.math.equal(y,0))
  mask = tf.cast(mask, dtype=loss.dtype)  
  loss = mask* loss
  loss = tf.reduce_mean(loss)
  return loss

buffer_size=32000
batch_size=150
Y_len = np.count_nonzero(y_train, axis=1)
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train, Y_len)).shuffle(buffer_size=buffer_size).batch(batch_size=batch_size)
valid_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val, np.count_nonzero(y_val, axis=1))).shuffle(buffer_size=buffer_size).batch(batch_size=batch_size)
optimizer = tf.keras.optimizers.Adam()

In [ ]:
for epoch in range(5000):
  avg_loss = 0
  training_step = 0
  for x_train2, y_train2, data_len in train_ds:
      with tf.GradientTape() as tape:
          loss = loss_function(model(x_train2), y_train2)
      grads = tape.gradient(loss, model.trainable_variables)
      optimizer.apply_gradients(grads_and_vars=zip(grads, model.trainable_variables))
      avg_loss += loss
      training_step += 1
  avg_loss /= training_step
  
  if (epoch + 1) % 10 == 0:
      avg_val_loss = 0
      val_training_step = 0
      for x_valid2, y_valid2, data_len2 in valid_ds:
          val_loss = loss_function(model(x_valid2), y_valid2)
          avg_val_loss += loss
          val_training_step += 1
      avg_val_loss /= val_training_step
      print('val_loss: {:.3f}'.format(avg_val_loss))
      print('Epoch: {:3}, tr_loss: {:.3f}'.format((epoch+1)/100, avg_loss))
      model.save(f'{path}/c_model.h5')

In [11]:
sample_num=12
y_pred = model.predict(np.expand_dims(X_test[sample_num],axis=0),verbose=0)
y_pred = np.argmax(y_pred, axis=-1) 
print(nl_tokenizer.decode(X_test[sample_num]))
print(code_tokenizer.decode(y_pred[0]))
print(code_tokenizer.decode(y_test[sample_num]))

find the area of a regular nk polygon inscribed in a circle of radius 1 however a regular nk polygon is take n points at equal intervals on the circumference the outermost figure connecting every k-1 points defined as for example a 52 polygon can be drawn as follows first take five equally spaceh points on a circle of radius 1 then connect each point every 2-1=1 the outermost figure is a regular 52 polygon
# include < stdio.h > # include < stdlib.h { int n for n ( i, 0, i, s ; long long " & ", ", ) ; for ( i ) a int = = 0 a ", ( " " " ( ( a " ( & < < ) scanf ( " < 0 ; i < a ) i - " + ) { " ( " ) n n n ) i ) ( ( 0 ) { ++ ( ( ) ) ( d ) ) ; ; d ; ; } ; } ) ( " ) 0 ) ; } ) 0 0 } } } } } } } ( " " ) ) ) ) ; ; ; ; ; ; ; ; ; ; ; ; ; ; ) ) ) ) ( ( ( * * * ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) d d d d d d d d d d d d d d d d d d d d d d d ) ) ) " " " " " " " " " " " " " " ) ) ) ) ; ; ; } } } } } } } 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [12]:
sample_num=0
y_pred = model.predict(np.expand_dims(X_train[sample_num],axis=0),verbose=0)
y_pred = np.argmax(y_pred, axis=-1)
print(nl_tokenizer.decode(X_train[sample_num]))
print(code_tokenizer.decode(y_pred[0]))
print(code_tokenizer.decode(y_train[sample_num]))

you are given an integer sequence of length n a1 an let us consider performing the following n operations on an empty sequence b the i-th operation is as follows becoming ai to the end of b reverse the order of the elements in b find the sequence b obtained after these n operations
i ; b [ 1 << 18 ] ; j ; j ( n, a ) { for ( i ~ scanf ( " %d ", & n ) - n % 2 ; ~ scanf ( " %d ", & a ) ; ) b [ ( n - i ) 2 + i % 2 * i ++ ] = a ; for for j j j < n ; ) printf ( " %d ", b [ j - 1 1 ) ; } } " " )
i ; b [ 1 << 18 ] ; j ; main ( n, a ) { for ( i = scanf ( " %d ", & n ) - n % 2 ; ~ scanf ( " %d ", & a ) ; ) b [ ( n - i ) 2 + i % 2 * i ++ ] = a ; for ( ; j ++ < n ; ) printf ( " %d ", b [ j - 1 ] ) ; }
